# docker

## 構築

https://github.com/ftakao2007/work

### 階層構造

* work
    * build
        * dockerhost : dockerhostを構築する(ubuntu,centos7)
            * コマンド例) ansible-playbook -i "127.0.0.1," -k -K -c paramiko centos7.yml
        * container
            * base : OS設定等基礎的なコンテナを作成する
                * コマンド例) sh create_container_centos7_base.sh
            * mysql : mysqlをコンテナにインストールする
                * コマンド例) sh create_container_mysql5.6_centos7.sh
            * (etc...)
        * docker-compose : docker-composeファイル置き場


1. work/build/dockerhostでdockerホストを構築 (初回のみ)
1. work/build/container/baseでOSの基本部分を構築する (初回のみ)
1. work/build/container/(各種サービス)でサービスのコンテナを構築し、dockerhubにpushする
1. work/build/docker-compose に先ほどdockerhubにpushしたコンテナを使うためのdocker-composeファイルを作成する

## mysqlをインストールする例

### コンテナの作成

In [24]:
%%bash
### ansibleを使うためにpython2系の環境を設定
export PYENV_VERSION="work"
#pyenv versions

work_dir="/home/jsd/work/build/container"
cd ${work_dir}/mysql
sh create_container_mysql5.6_centos7.sh

base_centos7
base_centos7

PLAY [containers] **************************************************************

TASK [mysql5.6.centos7 : MySQL Communityのrpmのインストール(MySQL5.6をインストールするため)] ******
changed: [base_centos7]

TASK [mysql5.6.centos7 : 必要なパッケージのインストール] **************************************
changed: [base_centos7] => (item=[u'mysql-community-client.x86_64', u'mysql-community-devel.x86_64', u'mysql-community-server.x86_64'])

TASK [mysql5.6.centos7 : my.cnfの設置] ********************************************
changed: [base_centos7]

TASK [mysql5.6.centos7 : MySQL データディレクトリを初期化し、システムテーブルを作成] *********************
changed: [base_centos7]

TASK [mysql5.6.centos7 : mysqlの起動と自動起動設定] **************************************
changed: [base_centos7]

TASK [mysql5.6.centos7 : rootユーザのパスワード変更] **************************************
changed: [base_centos7]

TASK [mysql5.6.centos7 : mysqlユーザの作成] ******************************************
changed: [base_centos7] => (item=127.0.0.0/255.255.255.0)
chan

Creating base_centos7


### コンテナをdockerhubにpush

```
### ログイン
$ docker login
================================================
Login with your Docker ID to push and pull images from Docker Hub. If you don't have a Docker ID, head over to https://hub.docker.com to create one.
Username (ftakao2007):
Password:
Login Succeeded
================================================

### コンテナを停止
$ docker stop base_centos7
================================================
base_centos7
================================================

### コミット
$ docker commit base_centos7 ftakao2007/mysql5.6_centos7:latest
================================================
sha256:b8b3619cf77682a92ec264528648d69a08c93a021ae9ff65cb8df34eeccfcec3
================================================

### dockerhubにpush
$ docker push ftakao2007/mysql5.6_centos7:latest
================================================
The push refers to a repository [docker.io/ftakao2007/mysql5.6_centos7]
775911eff0d1: Pushed
c98c19d806c4: Mounted from ftakao2007/centos7
99b28d9413e4: Mounted from ftakao2007/centos7
latest: digest: sha256:fd2a322cdbafb5d035b16855f64f46a1768ca1b8e029276f43d98759bde40455 size: 955
================================================

### ブラウザでアクセスして確認
https://hub.docker.com/r/ftakao2007/mysql5.6_centos7/
```

### docker-composeファイルの作成

```
### ディレクトリに移動
$ cd ../docker-compose

### docker-composeファイルの作成
$ vi docker-compose_mysql5.6_centos7.yml
================================================
mysql5.6_centos7:
   image: ftakao2007/mysql5.6_centos7:latest
   container_name: mysql5.6_centos7
   privileged: true
   tty: true
   ports:
     - "3306:3306"
     - "2222:22"
   command: sh -c 'exec /usr/sbin/init && systemctl restart sshd.service && systemctl restart mysqld.service'
================================================
```

In [33]:
%%bash

### イメージ、コンテナが無い状態の確認
docker images
docker ps -a

### docker-composeディレクトリに移動
work_dir="/home/jsd/work/build/docker-compose"
cd ${work_dir}

### コンテナの作成
docker-compose -f docker-compose_mysql5.6_centos7.yml up -d
sleep 60

### イメージ、コンテナの確認
docker images
docker ps -a

### mysql接続
mysql -ujoruri -p'pass' -h 127.0.0.1 -P 3306 -e "SELECT CURDATE();" 2> /dev/null

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
latest: Pulling from ftakao2007/mysql5.6_centos7
Digest: sha256:fd2a322cdbafb5d035b16855f64f46a1768ca1b8e029276f43d98759bde40455
Status: Downloaded newer image for ftakao2007/mysql5.6_centos7:latest
REPOSITORY                    TAG                 IMAGE ID            CREATED             SIZE
ftakao2007/mysql5.6_centos7   latest              b8b3619cf776        33 minutes ago      1.79GB
CONTAINER ID        IMAGE                                COMMAND                  CREATED              STATUS              PORTS                                          NAMES
6dbcd3e4d535        ftakao2007/mysql5.6_centos7:latest   "sh -c 'exec /usr/..."   About a minute ago   Up About a minute   0.0.0.0:3306->3306/tcp, 0.0.0.0:2222->22/tcp   mysql5.6_centos7
CURDATE()
2017-07-14


Pulling mysql5.6_centos7 (ftakao2007/mysql5.6_centos7:latest)...
Creating mysql5.6_centos7
